In [1]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
import scipy as sp
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [2]:
def simu_success(tag, params):
    # which task
    if tag == "Asso-CR":
        path = "../Data/simuS1_group1_design.pkl"
        nitems = 96
        test1_num = 40
    elif tag == "Pair-CR":
        path = "../Data/simuS1_group2_design.pkl"
        nitems = 176
        test1_num = 80
    elif tag == "Item-CR":
        path = "../Data/simuS1_group3_design.pkl"
        nitems = 136
        test1_num = 80

    # load stimuli
    with open(path, 'rb') as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    # df_study = df_study.loc[df_study.session < 200]
    # df_test = df_test.loc[df_test.session < 200]

    # load semantic matrix
    s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

    # update n_item in params to fit the study
    params.update(nitems_in_accumulator = nitems)
    # print(params)

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test ,s_mat)
    df_simu['test'] = df_test['test']
    df_simu = df_simu.merge(df_test,on=['session','test','test_itemno1','test_itemno2'])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort

        tmp_corr = df_simu.loc[df_simu.session == sess,"correct_ans"][test1_num:]
        corrid = np.searchsorted(tmp, tmp_corr)

        corr_fin = [f_in_dif[sess][i][id] for i, id in enumerate(corrid)]
        omax_fin = [np.max(np.delete(f_in_dif[sess][i], id)) for i, id in enumerate(corrid)]

        tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
        tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu['corr_fin'] = tmp_corr_fin
    df_simu['omax_fin'] = tmp_omax_fin

    return df_simu

In [3]:
def anal_perform(df_simu):

    # get correctness
    df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans

    # recognition performance
    df_recog = df_simu.query("test==1")
    hr_far = df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")
    z_hr_far = sp.stats.norm.ppf(hr_far)
    print("recognition: \n", hr_far)
    print("d_prime: ", z_hr_far[1].item() - z_hr_far[0].item())

    # cued recall performance
    df_cr = df_simu.query("test==2")
    print("cued recall: \n", df_cr.correct.mean())

    # analyze pair
    def get_pair(df_tmp):
        df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
        df_tmp_pair.columns = ["test1","test2"]
        df_tmp_pair.reset_index(inplace=True)
        return df_tmp_pair

    df_simu_p = df_simu.query("pair_idx >= 0")
    df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
    df_tab = pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize=False)
    print("contingency table: \n", df_tab)
    print("contingency table norm: \n", pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all'))

    # compute" Q
    def Yule_Q(A, B, C, D):
        return (A * D - B * C) / (A * D + B * C)
    
    # print("Q: ", Yule_Q(df_tab[1][1],df_tab[0][1],df_tab[1][0],df_tab[0][0]))
    print("Q: ", Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5))  # add 0.5

    return None

### Study Params

In [4]:
# study params should be the same across three groups
params = cmr.make_default_params()
params.update(
    beta_enc = 0.3,
    gamma_fc = 0.2,
    gamma_cf = 0.2,
    s_fc = 0.2,
    s_cf = 0.2,
    phi_s = 1,
    phi_d = 0.6,
    d_ass = 1,
    learn_while_retrieving = False,
    var_enc = 0.7,
    bad_enc_ratio = 0.3,
)

In [5]:
params.update(
    beta_cue = 0.4,
    beta_rec = 0.1,  # beta for retrieved item
    beta_distract = 0.1,
    # beta_rec_post = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.5,
    kappa = 0.01,
    lamb = 0.002,
    eta = 0.002,
    omega = 3,
    alpha = 1
)
params

{'beta_enc': 0.3,
 'beta_rec': 0.1,
 'beta_cue': 0.4,
 'beta_rec_post': 0.5,
 'beta_distract': 0.1,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.2,
 's_fc': 0.2,
 'kappa': 0.01,
 'eta': 0.002,
 'omega': 3,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.5,
 'd_ass': 1,
 'lamb': 0.002,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 0.7,
 'bad_enc_ratio': 0.3,
 'gamma_fc': 0.2,
 'gamma_cf': 0.2}

In [6]:
params_g1 = params.copy()
# params_g1.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.5,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g1

In [7]:
params_g2 = params.copy()
# params_g2.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.50,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g2

In [8]:
params_g3 = params.copy()
# params_g3.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.50,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g3

### Association - CR (Group1)

In [9]:
tag = "Asso-CR"
df_simu_g1 = simu_success(tag, params_g1)
df_simu_g1

CMR2 Time: 65.45386147499084


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,146,1542,1.0,0.010489,0.634740,1,BOLT,UTENSIL,1,0,34,-1.000000,-1.000000
1,0,147,1118,0.0,16.671247,0.266184,1,BOMB,PRAIRIE,1,1,0,-1.000000,-1.000000
2,0,1185,159,0.0,1.261298,0.395262,1,REPTILE,BOX,0,1,-1,-1.000000,-1.000000
3,0,358,1421,1.0,0.097278,0.523378,1,COMPASS,SWING,1,1,35,-1.000000,-1.000000
4,0,1201,1359,1.0,0.116974,0.514159,1,ROCK,SPRINKLE,1,0,12,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,999,1433,-1,155.0,2150.000000,0.576070,2,TAPE,-1,155,0,26,-0.454103,-0.563037
59996,999,1463,-1,986.0,2780.000000,0.429590,2,TICK,-1,700,1,13,-0.709775,-0.588467
59997,999,990,-1,1317.0,1920.000000,0.574363,2,OVAL,-1,1317,1,28,-0.447845,-0.578299
59998,999,695,-1,400.0,2680.000000,0.402929,2,HEAD,-1,52,0,19,-0.557148,-0.586781


In [10]:
csim_two = df_simu_g1.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.325846
1    0.427470
Name: csim, dtype: float64

In [11]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.3766579923211597

In [12]:
df_simu_g1['s_resp'] = df_simu_g1.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [13]:
anal_perform(df_simu_g1)

recognition: 
              Yes rate
correct_ans          
0              0.2712
1              0.6468
d_prime:  0.9858831240350268
cued recall: 
 0.2822
contingency table: 
 test1   0.0   1.0
test2            
0.0    7047  7309
1.0      17  5627
contingency table norm: 
 test1      0.0      1.0
test2                  
0.0    0.35235  0.36545
1.0    0.00085  0.28135
Q:  0.9935700642327542


### Pair - CR (Group2)

In [14]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params_g2)
df_simu_g2

CMR2 Time: 168.94850635528564


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,1109,898,1.0,0.028645,0.584507,1,PORPOISE,MINT,1,1,32,-1.000000,-1.000000
1,0,661,567,0.0,23.437260,0.249152,1,GRILL,FINGER,0,-1,-1,-1.000000,-1.000000
2,0,1086,732,1.0,0.082826,0.531419,1,PLATE,ICING,1,1,39,-1.000000,-1.000000
3,0,249,720,0.0,1.574808,0.384162,1,CARROT,HOSE,1,1,13,-1.000000,-1.000000
4,0,1385,340,1.0,0.096813,0.523617,1,STONE,COD,1,0,28,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,934,-1,1529.0,3100.000000,0.436705,2,MUSHROOM,-1,1000,0,10,-0.643822,-0.583432
119996,999,1015,-1,288.0,3130.000000,0.512454,2,PARROT,-1,288,0,12,-0.523479,-0.560571
119997,999,1619,-1,396.0,3550.000000,0.464976,2,WOLF,-1,1185,1,36,-3.272293,-0.552493
119998,999,355,-1,141.0,2860.000000,0.472053,2,COMMITTEE,-1,755,1,18,-3.410169,-0.571195


In [15]:
csim_two = df_simu_g2.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.248200
1    0.448983
Name: csim, dtype: float64

In [16]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.34859162230118107

In [17]:
df_simu_g2['s_resp'] = df_simu_g2.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [18]:
anal_perform(df_simu_g2)

recognition: 
              Yes rate
correct_ans          
0            0.079150
1            0.785525
d_prime:  2.201801890668122
cued recall: 
 0.24125
contingency table: 
 test1   0.0    1.0
test2             
0.0    8558  21792
1.0      21   9629
contingency table norm: 
 test1       0.0       1.0
test2                    
0.0    0.213950  0.544800
1.0    0.000525  0.240725
Q:  0.9886939196398058


### Item - CR (Group3)

In [19]:
tag = "Item-CR"
df_simu_g3 = simu_success(tag, params_g3)
df_simu_g3

CMR2 Time: 134.27810621261597


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,1292,-1,1.0,1.605324,0.383202,1,SHORTCAKE,1,0,14,-1.000000,-1.000000
1,0,1052,-1,1.0,0.308513,0.465668,1,PERSON,1,0,39,-1.000000,-1.000000
2,0,135,-1,1.0,0.998423,0.406948,1,BLANKET,1,0,19,-1.000000,-1.000000
3,0,1365,-1,1.0,361.483795,0.112358,1,STAFF,0,-1,-1,-1.000000,-1.000000
4,0,760,-1,1.0,151.365845,0.155884,1,JOURNAL,0,-1,-1,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,1268,-1,1419.0,3860.000000,0.387505,2,SENATOR,157,0,37,-3.444944,-0.627571
119996,999,537,-1,546.0,3100.000000,0.428295,2,ENVELOPE,546,1,3,-0.600067,-0.642470
119997,999,361,-1,364.0,4080.000000,0.378838,2,CONCRETE,260,0,28,-3.454541,-0.646020
119998,999,395,-1,356.0,2710.000000,0.498961,2,COWBOY,356,1,31,-0.548027,-0.649173


In [20]:
csim_two = df_simu_g3.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.236955
1    0.330744
Name: csim, dtype: float64

In [21]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.28384948908480945

In [22]:
df_simu_g3['s_resp'] = df_simu_g3.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [23]:
anal_perform(df_simu_g3)

recognition: 
              Yes rate
correct_ans          
0            0.266225
1            0.731025
d_prime:  1.2401863709859529
cued recall: 
 0.2608
contingency table: 
 test1   0.0    1.0
test2             
0.0    9994  19574
1.0     765   9667
contingency table norm: 
 test1       0.0       1.0
test2                    
0.0    0.249850  0.489350
1.0    0.019125  0.241675
Q:  0.7314792182723583


## No Variability

In [24]:
params.update(
    var_enc = 1,
    bad_enc_ratio = 1,
)
params

{'beta_enc': 0.3,
 'beta_rec': 0.1,
 'beta_cue': 0.4,
 'beta_rec_post': 0.5,
 'beta_distract': 0.1,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.2,
 's_fc': 0.2,
 'kappa': 0.01,
 'eta': 0.002,
 'omega': 3,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.5,
 'd_ass': 1,
 'lamb': 0.002,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.2,
 'gamma_cf': 0.2}

In [25]:
params_g1 = params.copy()
# params_g1.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.5,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g1

In [26]:
params_g2 = params.copy()
# params_g2.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.50,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g2

In [27]:
params_g3 = params.copy()
# params_g3.update(
#     beta_cue = 0.4,
#     beta_rec = 0,
#     beta_rec_new = 0.1, # beta for retrieved item
#     beta_rec_post = 0.1,
#     c_thresh = 0.01,
#     c_thresh_ass = 0.50,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params_g3

### Association - CR (Group1)

In [28]:
tag = "Asso-CR"
df_simu_g1 = simu_success(tag, params_g1)
df_simu_g1

CMR2 Time: 64.28631258010864


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,146,1542,1.0,0.010489,0.634740,1,BOLT,UTENSIL,1,0,34,-1.000000,-1.000000
1,0,147,1118,0.0,1.457310,0.388039,1,BOMB,PRAIRIE,1,1,0,-1.000000,-1.000000
2,0,1185,159,0.0,1.545035,0.385116,1,REPTILE,BOX,0,1,-1,-1.000000,-1.000000
3,0,358,1421,1.0,0.105787,0.519185,1,COMPASS,SWING,1,1,35,-1.000000,-1.000000
4,0,1201,1359,1.0,0.119928,0.512912,1,ROCK,SPRINKLE,1,0,12,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,999,1433,-1,155.0,1880.000000,0.579363,2,TAPE,-1,155,0,26,-0.450981,-0.557330
59996,999,1463,-1,700.0,2310.000000,0.549580,2,TICK,-1,700,1,13,-0.499588,-0.559465
59997,999,990,-1,1317.0,2000.000000,0.571492,2,OVAL,-1,1317,1,28,-0.450605,-0.563979
59998,999,695,-1,648.0,2660.000000,0.395878,2,HEAD,-1,52,0,19,-0.553884,-0.560024


In [29]:
csim_two = df_simu_g1.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.367408
1    0.504666
Name: csim, dtype: float64

In [30]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.43603700909763576

In [31]:
df_simu_g1['s_resp'] = df_simu_g1.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [32]:
anal_perform(df_simu_g1)

recognition: 
              Yes rate
correct_ans          
0             0.15850
1             0.88385
d_prime:  2.195096914173958
cued recall: 
 0.4197
contingency table: 
 test1   0.0   1.0
test2            
0.0    2142  9464
1.0     181  8213
contingency table norm: 
 test1      0.0      1.0
test2                  
0.0    0.10710  0.47320
1.0    0.00905  0.41065
Q:  0.8221293036083623


### Pair - CR (Group2)

In [33]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params_g2)
df_simu_g2

CMR2 Time: 148.68788862228394


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,1109,898,1.0,0.028645,0.584507,1,PORPOISE,MINT,1,1,32,-1.000000,-1.000000
1,0,661,567,0.0,23.437260,0.249152,1,GRILL,FINGER,0,-1,-1,-1.000000,-1.000000
2,0,1086,732,1.0,0.082826,0.531419,1,PLATE,ICING,1,1,39,-1.000000,-1.000000
3,0,249,720,1.0,0.052403,0.554308,1,CARROT,HOSE,1,1,13,-1.000000,-1.000000
4,0,1385,340,1.0,0.093647,0.525280,1,STONE,COD,1,0,28,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,934,-1,1619.0,3220.000000,0.468692,2,MUSHROOM,-1,1000,0,10,-3.434815,-0.523970
119996,999,1015,-1,1486.0,2180.000000,0.527477,2,PARROT,-1,288,0,12,-0.508080,-0.465576
119997,999,1619,-1,628.0,2670.000000,0.513072,2,WOLF,-1,1185,1,36,-3.246902,-0.497140
119998,999,355,-1,1074.0,2900.000000,0.433863,2,COMMITTEE,-1,755,1,18,-3.412698,-0.573460


In [34]:
csim_two = df_simu_g2.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.248298
1    0.518710
Name: csim, dtype: float64

In [35]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.38350435871999544

In [36]:
df_simu_g2['s_resp'] = df_simu_g2.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [37]:
anal_perform(df_simu_g2)

recognition: 
              Yes rate
correct_ans          
0            0.022275
1            0.975225
d_prime:  3.972707054086244
cued recall: 
 0.384725
contingency table: 
 test1  0.0    1.0
test2            
0.0    950  23661
1.0     41  15348
contingency table norm: 
 test1       0.0       1.0
test2                    
0.0    0.023750  0.591525
1.0    0.001025  0.383700
Q:  0.8738718033994808


### Item - CR (Group3)

In [38]:
tag = "Item-CR"
df_simu_g3 = simu_success(tag, params_g3)
df_simu_g3

CMR2 Time: 121.33141255378723


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,corr_fin,omax_fin
0,0,1292,-1,1.0,0.350559,0.459280,1,SHORTCAKE,1,0,14,-1.000000,-1.000000
1,0,1052,-1,1.0,0.283493,0.469897,1,PERSON,1,0,39,-1.000000,-1.000000
2,0,135,-1,1.0,0.871508,0.413745,1,BLANKET,1,0,19,-1.000000,-1.000000
3,0,1365,-1,1.0,354.443207,0.113341,1,STAFF,0,-1,-1,-1.000000,-1.000000
4,0,760,-1,1.0,148.892548,0.156707,1,JOURNAL,0,-1,-1,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,1268,-1,157.0,2040.000000,0.581104,2,SENATOR,157,0,37,-0.450800,-0.579474
119996,999,537,-1,1506.0,3550.000000,0.413148,2,ENVELOPE,546,1,3,-0.597005,-0.579906
119997,999,361,-1,260.0,2600.000000,0.562416,2,CONCRETE,260,0,28,-0.474917,-0.585139
119998,999,395,-1,356.0,2660.000000,0.503905,2,COWBOY,356,1,31,-0.543548,-0.582654


In [39]:
csim_two = df_simu_g3.query("test==1").groupby("correct_ans").csim.mean()
csim_two

correct_ans
0    0.239780
1    0.365604
Name: csim, dtype: float64

In [40]:
opt_thresh = np.mean(csim_two)
opt_thresh

0.30269186140199306

In [41]:
df_simu_g3['s_resp'] = df_simu_g3.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

In [42]:
anal_perform(df_simu_g3)

recognition: 
              Yes rate
correct_ans          
0             0.19490
1             0.83115
d_prime:  1.8186997653967534
cued recall: 
 0.4275
contingency table: 
 test1   0.0    1.0
test2             
0.0    5606  17294
1.0    1148  15952
contingency table norm: 
 test1      0.0      1.0
test2                  
0.0    0.14015  0.43235
1.0    0.02870  0.39880
Q:  0.6365479211839657
